In [129]:
import pandas as pd
from pathlib import Path  
import sys
import sqlite3

CUT_OF_UNIQUE_VALUES=40

In [130]:
data_filtered_cedd=pd.read_csv("cleaned_data_with_binary_dead.csv")
df_counts  = pd.DataFrame(columns = ['col', 'count'])
for col in data_filtered_cedd:
    no_of_unique_values=data_filtered_cedd[col].nunique()
    df_counts.loc[len(df_counts)]=[col,no_of_unique_values]
#     if( not col.startswith( 'Unnamed' ) && no_of_unique_values<20):
#         print("%s--->%s"%(col,))
df_counts = df_counts.sort_values(by=['count'], ascending=False)
display(df_counts.head(30))

count=0

for col in data_filtered_cedd:
    no_of_unique_values=data_filtered_cedd[col].nunique()
    if( not col.startswith( 'Unnamed' ) and no_of_unique_values<40):
        count =count+1
        #data_filtered_cedd[col].fillna("imputed",inplace=True)
        data_filtered_cedd[col]=pd.Categorical(data_filtered_cedd[col])
                
data_filtered_cedd.dtypes.value_counts()



count


    



C:\Users\sjyhl\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (75,83,88,136) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,col,count
0,Unnamed: 0,28122
2,Unnamed: 0.1.1,28122
1,Unnamed: 0.1,28122
29,ceo_id,15744
28,epi_id,7432
4,pa_id,5930
13,epi_SocDate,1056
22,icd_code,967
23,icd_code_description,965
14,epi_StartOfEpisode,833


133

### New Binary Column for label and drop alive patients with less than 365 days of history

In [131]:
from datetime import timedelta
import numpy as np

##### adding our binary column to denote if patient is dead
df = data_filtered_cedd
df['patient_deceased'] = np.where((df['DaysToDeath'] <= 365) | (df['dcc_desc'] == 'PATIENT EXPIRED'), 1, 0)

In [132]:
##### defining our year threshold, and converting neccesary col's to datetimes

threesixtyfive = timedelta(days = 365)

df['epi_SocDate'] = pd.to_datetime(df['epi_SocDate'])
df['epi_EndOfEpisode'] = pd.to_datetime(df['epi_EndOfEpisode'])
df['M0906_DC_TRAN_DTH_DT'] = pd.to_datetime(df['M0906_DC_TRAN_DTH_DT'].fillna(df['epi_SocDate']))

##### get first 3 chars of ICD value
df['icd_group'] = df['icd_code'].str[:3]

C:\Users\sjyhl\AppData\Local\Temp/ipykernel_13280/1806077930.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['icd_group'] = df['icd_code'].str[:3]


In [133]:
##### dropping all 'alive' patients with LESS than a year of history

for index, row in df.iterrows():
    if row['patient_deceased'] == 0 and row['epi_EndOfEpisode'] - row['epi_SocDate'] < threesixtyfive:
        df.drop(index, axis = 0, inplace = True)

In [134]:
##### dropping all 'dead' patients with MORE than a year of history

for index, row in df.iterrows():
    if row['patient_deceased'] == 1 and row['epi_EndOfEpisode'] - row['epi_SocDate'] > threesixtyfive:
        row['patient_deceased'] == 0

In [135]:
len(df)

12981

In [136]:
df["M1056_PVX_RSN_NOT_RCVD_AGNCY"].cat.categories
df.groupby(['M1056_PVX_RSN_NOT_RCVD_AGNCY']).size()

df["M1200_VISION"].cat.categories
df.groupby(['M1200_VISION']).size()

M1200_VISION
0 - NORMAL VISION: SEES ADEQUATELY IN MOST SITUATIONS; CAN SEE MEDICATION LABELS, NEWSPRINT                                                                        5242
1 - PARTIALLY IMPAIRED: CANNOT SEE MEDICATION LABELS OR NEWSPRINT, BUT CAN SEE OBSTACLES IN PATH, AND THE SURROUNDING LAYOUT; CAN COUNT FINGERS AT ARM'S LENGTH    1847
2 - SEVERELY IMPAIRED: CANNOT LOCATE OBJECTS WITHOUT HEARING OR TOUCHING THEM OR PATIENT NONRESPONSIVE                                                               95
dtype: int64

### Drop columns with no meaning

In [137]:
df = df.drop(columns=['CreatedDate', 'CreatedBy', 'UpdateDate', 'UpdateBy', 'ActiveFlag'])

In [138]:
df['DaysToDeath']

40       194.0
41       194.0
42       194.0
43       194.0
76        12.0
         ...  
28117      2.0
28118      2.0
28119      2.0
28120      2.0
28121      2.0
Name: DaysToDeath, Length: 12981, dtype: float64

### Find columns with name ID

In [139]:
[col for col in df.columns if 'id' in col]

['pa_id', 'epi_dcrid', 'epi_id', 'ceo_id']

In [140]:
[col for col in df.columns if 'Id' in col]

['DispositionId']

### Drop ID columns... this could give model too much hint

In [141]:
df = df.drop(columns=['DispositionId', 'pa_id', 'epi_dcrid', 'epi_id', 'ceo_id'])

### Drop records with DaysToDeath = 0

In [142]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df.select_dtypes(include=numerics)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,year_born,epi_height,epi_weight,DaysToDeath,RoughAge,patient_deceased
40,40,40,40,1930,NaN,NaN,194.0,NaN,1
41,41,41,41,1930,NaN,NaN,194.0,NaN,1
42,42,42,42,1930,NaN,NaN,194.0,NaN,1
43,43,43,43,1930,NaN,NaN,194.0,NaN,1
76,76,76,76,1966,NaN,NaN,12.0,NaN,1
...,...,...,...,...,...,...,...,...,...
28117,28117,28117,28117,1940,NaN,NaN,2.0,82.0,1
28118,28118,28118,28118,1940,NaN,NaN,2.0,82.0,1
28119,28119,28119,28119,1940,NaN,NaN,2.0,82.0,1
28120,28120,28120,28120,1940,NaN,NaN,2.0,82.0,1


### Drop columns that are giving hints as response variables (like cedd_dateofdeath).

In [143]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [144]:
### Also drop dcc_desc and dcr_desc

In [145]:
df.drop(['M2200_THER_NEED_NBR', 'M1322_NBR_PRSULC_STG1', 'M0090_INFO_COMPLETED_DT', 'ceo_OASIScorrectionnum', 'cedd_dateofdeath', 'ceas_ascode', 'epi_DischargeDate', 'epi_EndOfEpisode', 'epi_StartOfEpisode', 'epi_SocDate', 'dcr_desc', 'dcc_desc', 'icd_code_description'], inplace = True, axis = 1)

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np

imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='imputed')
idf = pd.DataFrame(imputer.fit_transform(df))
idf.columns=df.columns
# idf.index=df.index

#
df = idf
for col in df:
    no_of_unique_values=df[col].nunique()
    if( not col.startswith( 'Unnamed' ) and no_of_unique_values<40):        
        df[col]=pd.Categorical(df[col])

In [147]:
df.dtypes

year_born                          object
pa_race                          category
pa_gender                        category
pa_maritalstatus                 category
epi_AdmitType                    category
                                   ...   
M2401_INTRVTN_SMRY_PRSULC_WET    category
M2410_INPAT_FACILITY             category
M2420_DSCHRG_DISP                category
patient_deceased                 category
icd_group                          object
Length: 131, dtype: object

#### detect columns that contain numeric value when it shouldn't be. (This runs quite a long time)

In [148]:
for column_name in df.columns:
    if column_name != 'year_born':
        for index, row in df.iterrows():
            if isinstance(row[column_name], (int, float)):
                print(column_name)
                df[column_name] = df[column_name].astype(str)
                break

pa_race
pa_maritalstatus
epi_height
epi_weight
DaysToDeath
RoughAge
M1311_NBR_ULC_SOCROC_STG2_A2
M1311_NBR_ULC_SOCROC_STG3_B2
M1311_NBR_ULC_SOCROC_STG4_C2
M1311_NSTG_CVRG_SOCROC_E2
M1311_NSTG_DEEP_TSUE_SOCROC_F2
M1311_NSTG_DRSG_SOCROC_D2
patient_deceased


In [149]:
# Use Ordinal Encoder for all numeric type category

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(df)
enc.categories_

[array([1909, 1914, 1915, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924,
        1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935,
        1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946,
        1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957,
        1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968,
        1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
        1980, 1981, 1982, 1983, 1984, 1985, 1988, 1989, 1990, 1993, 1996,
        1999, 2011], dtype=object),
 array(['1.0', '2.0', '3.0', '5.0', '6.0', 'imputed'], dtype=object),
 array(['FEMALE', 'MALE  '], dtype=object),
 array(['1.0', '2.0', '25000.0', '3.0', '4.0', '5.0', '6.0', '8.0',
        'imputed'], dtype=object),
 array(['NEW ADMISSION', 'READMISSION', 'RECERTIFICATION'], dtype=object),
 array(['5.0', '5.3', '5.6', '55.0', '56.25', '57.0', '59.0', '59.8',
        '60.0', '60.5', '60.98', '61.0', '61.2', '61.96', '61.97', '62.0',
        '

In [150]:
xdf = enc.transform(df) # transform using encoder

In [151]:
df2 = pd.DataFrame(xdf, columns=df.columns) # numpy to dataframe

In [152]:
X = df2.iloc[:,:-1]

In [153]:
y = df2['DaysToDeath']

In [160]:
X.drop('DaysToDeath', 1, inplace=True)

C:\Users\sjyhl\AppData\Local\Temp/ipykernel_13280/1373080911.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X.drop('DaysToDeath', 1, inplace=True)


KeyError: "['DaysToDeath'] not found in axis"

In [162]:
X.drop('patient_deceased', 1, inplace=True)

C:\Users\sjyhl\AppData\Local\Temp/ipykernel_13280/2774115035.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X.drop('patient_deceased', 1, inplace=True)
C:\Users\sjyhl\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [163]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

### Feature Selection

In [164]:
from sklearn.linear_model import LassoCV

# Lasso with 5 fold cross-validation
model = LassoCV(cv=5, random_state=0, max_iter=10000)

# Fit model
model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=10000, random_state=0)

model.alpha_

3.3430373413801644

In [165]:
from sklearn.linear_model import Lasso
lasso_best = Lasso(alpha=model.alpha_)
lasso_best.fit(X_train, y_train)
print([coef for coef in list(zip(lasso_best.coef_, X)) if abs(coef[0]) > 0])

[(0.46048511234250544, 'year_born'), (-0.1486884227047609, 'pa_race'), (6.197476311926682, 'epi_AdmitType'), (-0.1190917929449747, 'epi_weight'), (0.15037783011711747, 'ceo_HIPPS'), (-0.6010260272049094, 'ceo_HHRG'), (0.019222752133072188, 'icd_code'), (1.3812723262961617, 'RoughAge'), (-0.23782475329998612, 'M0100_ASSMT_REASON'), (-0.0442685624778846, 'M0906_DC_TRAN_DTH_DT'), (-0.4289026441236098, 'M1060_HEIGHT_A'), (0.05193019077990979, 'M1060_WEIGHT_B'), (-0.7239225745664273, 'M1311_NSTG_DEEP_TSUE_F1'), (-1.5485718524421717, 'M1400_WHEN_DYSPNEIC'), (0.549218226968907, 'M1620_BWL_INCONT'), (0.10289556709844966, 'M1710_WHEN_CONFUSED'), (-0.2705914071541215, 'M2410_INPAT_FACILITY')]


In [166]:
[coef[1] for coef in list(zip(lasso_best.coef_, X)) if abs(coef[0]) > 0]

['year_born',
 'pa_race',
 'epi_AdmitType',
 'epi_weight',
 'ceo_HIPPS',
 'ceo_HHRG',
 'icd_code',
 'RoughAge',
 'M0100_ASSMT_REASON',
 'M0906_DC_TRAN_DTH_DT',
 'M1060_HEIGHT_A',
 'M1060_WEIGHT_B',
 'M1311_NSTG_DEEP_TSUE_F1',
 'M1400_WHEN_DYSPNEIC',
 'M1620_BWL_INCONT',
 'M1710_WHEN_CONFUSED',
 'M2410_INPAT_FACILITY']

### Filter X

In [168]:
X = X[['year_born',
 'pa_race',
 'epi_AdmitType',
 'epi_weight',
 'ceo_HIPPS',
 'ceo_HHRG',
 'icd_code',
 'RoughAge',
 'M0100_ASSMT_REASON',
 'M0906_DC_TRAN_DTH_DT',
 'M1060_HEIGHT_A',
 'M1060_WEIGHT_B',
 'M1311_NSTG_DEEP_TSUE_F1',
 'M1400_WHEN_DYSPNEIC',
 'M1620_BWL_INCONT',
 'M1710_WHEN_CONFUSED',
 'M2410_INPAT_FACILITY']]

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

### Linear Regression using selected features

In [170]:
from sklearn.linear_model import LinearRegression

In [171]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [172]:
y_pred = reg.predict(X_test)

In [173]:
import matplotlib.pyplot as plt

In [174]:
from sklearn.metrics import mean_squared_error, r2_score

# The coefficients
print("Coefficients: \n", reg.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination (R-squared): %.2f" % r2_score(y_test, y_pred))



Coefficients: 
 [ 0.47563482 -1.72918986 11.29574895 -0.1310009   0.15986021 -0.74870516
  0.01597855  1.39635935 -1.85306912 -0.0400754  -0.59416072  0.06165001
 -0.9332571  -6.68411997  2.18633551  0.59718485 -2.3325444 ]
Mean squared error: 11955.22
Coefficient of determination: 0.07


0         99.0
1         99.0
2         99.0
3         99.0
4         31.0
         ...  
12976    104.0
12977    104.0
12978    104.0
12979    104.0
12980    104.0
Name: DaysToDeath, Length: 12981, dtype: float64